# Seleção dos hyperparâmetros 

Hyperparâmetros são os parâmetros que não são aprendidos no processo de treinamento da rede neural. São parâmetros tanto do modelo, como número e tipo de camadas, 
número de neurônios ou canais em cada camada, como parâmetros do processo de otimização, tais como tipo do otimizador, taxa de aprendizado (learning rate), tamanho
do mini-batch, entre outros.

Uma opção é disparar um processo automático de busca no espaço de hyperparâmetros para buscar o melhor índice de validação cruzada. 

Normalmente a busca neste espaço de hyperparâmetros consiste de:
- o modelo da rede;
- o espaço de hyperparâmetros;
- o método de busca e amostragem neste espaço;
- o esquema de validação cruzada; e
- uma função alvo (*score function*)



<img src='../figures/model_selection.png', width=600></img>

## Validação cruzada

<img src='../figures/cross_validation.png', width=600></img>

## Objetivos desse experimento

Neste experimento apresentamos implementações simplificadas de dois métodos populares de busca de hiperparâmetros, a busca em grade e a busca randomizada.

Este experimento utiliza:
- **modelo da rede:** fixa - Fully connected, já usada anteriormente

- **espaço de hiperparâmetros:** variando learning rate e weight decay

- **métodos de busca:** 

  - RandomizedSearch, onde o número de iterações é especificado
  - GridSearch, onde o número de iterações é dado pelas combinações de valores dos parâmetros

- **validação cruzada:** n. de folds especificado

- **função alvo:** mse loss

## Importação dos pacotes tradicionais

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import os, sys
import time
import numpy as np
import itertools

import scipy.stats as st
import numpy.random as nr

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

import torchvision as tv
import lib.pytorch_trainer as ptt

use_gpu = torch.cuda.is_available()
print('GPU available:', use_gpu)

np.set_printoptions(precision=3)

GPU available: True


## Carregamento dos dados - Boston Housing

In [2]:
datain = np.load('../data/boston_housing_normalize.npz')

x, y = datain['Xtra'], datain['ytra']

n_samples, n_attributes = x.shape

x_train = torch.FloatTensor(x)
y_train = torch.FloatTensor(y)

train_ds = TensorDataset(x_train, y_train)
print('Número de amostras no dataset (treino):', len(train_ds))


Número de amostras no dataset (treino): 506


### Sampler

In [3]:
from torch.utils.data.sampler import Sampler

class MySampler(Sampler):
    """ Esta é uma classe auxiliar que auxilia no processo de obtenção de amostras
    de um dataset. Trabalha com os índices de um subconjunto dessas amostras para
    a implementação da valiação cruzada. Um objeto dessa classe é passado ao construtor 
    do dataloader.
    """
    def __init__(self, indexes):
        self.indexes = indexes

    def __iter__(self):
        return iter(self.indexes[torch.randperm(len(self.indexes)).long()])

    def __len__(self):
        return len(self.indexes)


## Definição da rede neural

In [4]:
class Model(nn.Module):
    def __init__(self, n_attributes=n_attributes):
        super(Model, self).__init__()
        self.layer1 = nn.Linear(n_attributes, 64)
        self.ativ1  = nn.SELU()
        self.layer2 = nn.Linear(64, 64)
        self.ativ2  = nn.SELU()
        self.layer3 = nn.Linear(64, 64)
        self.ativ3  = nn.SELU()
        self.layer4 = nn.Linear(64, 1)

    def forward(self, x):
        x = self.layer1(x)
        x = self.ativ1(x)
        x = nn.functional.dropout(x)
        x = self.layer2(x)
        x = self.ativ2(x)
        x = nn.functional.dropout(x)
        x = self.layer3(x)
        x = self.ativ3(x)
        x = nn.functional.dropout(x)
        x = self.layer4(x)
        return x


## Implementação das classes de busca

### Funcionalidade básica

In [5]:
class MyBaseSearch(object):
    """ Classe base para a busca aleatória e em grade
    """
    def __init__(self, params, num_splits):
        if params['model_class'] is None:
            raise Exception('')
        self.parameters = params
        self.n_splits = num_splits
        
    def do_cross_validation(self, p, dataset, verbose):
        """ Implementa a valiação cruzada. Retorna os scores de valiação para 
        cada treinamento realizado.
        """
        split_scores = []
        for train_dloader, valid_dloader in self.gen_data_loaders(dataset, 
                                                                  self.n_splits, 
                                                                  p['batch_size']):
            tt = time.time()
            trainer = self.make_trainer(p, verbose=verbose)
            trainer.fit_loader(p['num_epochs'], train_dloader, valid_dloader)
            trainer.load_state(self.state_fn)
            score = trainer.metrics['valid']['losses'][-1]
            # metrics = trainer.evaluate_loader(valid_dloader, verbose=0)
            # score = metrics['losses']
            split_scores.append(score)
            print('{: 3.5f}'.format(score), end=' ')
        return split_scores
            
    @staticmethod
    def gen_data_loaders(dataset, n_splits, batch_size):
        """ Gerador que divide o dataset e retorna os dataloaders para cada 
        split de validação.
        """
        n_total = len(dataset)
        split_size = n_total // n_splits
        indices = torch.arange(n_total)
        split = torch.LongTensor([n_splits - 1 for _ in indices])
        split[:n_splits * split_size] = torch.arange(n_splits * split_size).long() / split_size
        for i in range(n_splits):
            vii = indices[split == i].long()
            tii = indices[split != i].long()
            train_dloader = DataLoader(dataset, batch_size=batch_size, sampler=MySampler(tii))
            valid_dloader = DataLoader(dataset, batch_size=batch_size, sampler=MySampler(vii))
            yield train_dloader, valid_dloader
    
    def make_trainer(self, p, verbose):
        """ Constroi um treinador com os parâmetros especificados no dicionário 'p'.
        """
        callbacks = [ptt.ModelCheckpoint(self.state_fn, reset=True, verbose=0)]
        if verbose > 0:
            callbacks += [ptt.PrintCallback()]
        self.model = p['model_class']()
        loss_fn = self.make_criterion(p)
        optimizer = self.make_optimizer(p)
        trainer = ptt.DeepNetTrainer(model=self.model, criterion=loss_fn, 
                                     optimizer=optimizer, callbacks=callbacks)
        return trainer
        
    def make_optimizer(self, p):
        """ Constroi um otimizador com base nos parâmetros 'p'.
        """
        if p['optim_class'] == 'Adam':
            optimz = optim.Adam(self.model.parameters(), lr=p['optim_lr'], 
                                weight_decay=p['optim_weight_decay'])
        elif p['optim_class'] == 'SGD':
            optimz = optim.SGD(self.model.parameters(), lr=p['optim_lr'], 
                               momentum=p['optim_momentum'], 
                               weight_decay=p['optim_weight_decay'], nesterov=True)
        elif p['optim_class'] == 'RMSprop':
            optimz = optim.RMSprop(self.model.parameters(), lr=p['optim_lr'], alpha=p['optim_alpha'],
                                   weight_decay=p['optim_weight_decay'])
        else:
            raise Exception("A ser implementado...")
        return optimz
    
    def make_criterion(self, p):
        """ Constroi uma função de custo com base nos parâmetros 'p'.
        """
        if p['loss_class'] == 'CrossEntropyLoss':
            loss_fn = nn.CrossEntropyLoss()
        elif p['loss_class'] == 'MSELoss':
            loss_fn = nn.MSELoss()
        else:
            raise Exception("A ser implementado...")
        return loss_fn
    
    def show_results(self):
        self.mean_scores = torch.FloatTensor(self.scores).mean(1)
        self.best_loss, self.best_index = [x[0] for x in self.mean_scores.min(0)]
        self.best_trainer = self.make_trainer(self.parameter_sets[self.best_index], verbose=0)
        print('\nBest parameter set from iteraction {} with loss = {:.5f}:'.format(self.best_index, self.best_loss))
        for p, v in self.parameter_sets[self.best_index].items():
            print('    {:20s}: {}'.format(p, v))


### Randomized Search

In [6]:
# param_distributions = {
#     'model_class':        None,
#     'loss_class':         ['MSELoss'],
#     'optim_class':        ['RMSprop'],
#     'optim_lr':           st.uniform(0.0005, 0.0015),     # range: 0.0005-0.002
#     'optim_weight_decay': st.uniform(0.001, 0.009),       # 0.001-0.01
#     'optim_alpha':        st.uniform(0.7, 0.29),          # 0.7-0.99
#     'batch_size':         [30],
#     'num_epochs':         [100],
# }

class MyRandomizedSearch(MyBaseSearch):
    """ Classe que implementa a busca randômica.
    """
    def __init__(self, params, num_splits=3, num_iter=10):
        super().__init__(params, num_splits)
        self.n_iteractions = num_iter
        self.state_fn = '/data/models/model_selection_random'
        
    def fit(self, dataset, verbose=False):
        """ Para cada iteração, amostra o espaço de parâmetros e executa uma
        validação cruzada.
        """
        self.parameter_sets = []
        self.scores = []
        for it in range(self.n_iteractions):
            print('Iteraction {:2d}:'.format(it), end=' ')
            t0 = time.time()
            p = self.sample_parameter_space()
            self.parameter_sets.append(p)
            
            split_scores = self.do_cross_validation(p, dataset, verbose)
            self.scores.append(split_scores)
            print('[{:.1f} s]'.format(time.time() - t0))
        self.show_results()
            
    def sample_parameter_space(self):
        """ Amostra cada parâmetro em 'self.parameters'. A especificação do 
        espaço é feita através de valores discretos em uma lista (que serão
        sorteados) ou de uma distribuição contínua (objeto do scipy.stats 
        com um método 'rvs') que será amostrada.
        """
        pars = dict()
        for p, v in self.parameters.items():
            if type(v) == list:
                pars[p] = v[0] if len(v) == 1 else v[nr.randint(0, len(v))]
            elif getattr(v, 'rvs', None) is not None:
                pars[p] = v.rvs()
            else:
                raise Exception('Unknown par dist type')
        return pars
    


### Grid Search

In [7]:
# param_distributions = {
#     'model_class':        None,
#     'loss_class':         ['MSELoss'],
#     'optim_class':        ['RMSprop'],
#     'optim_lr':           [0.0005, 0.001, 0.002],
#     'optim_weight_decay': [0.001, 0.005, 0.01],
#     'optim_alpha':        [0.7, 0.99],
#     'batch_size':         [30],
#     'num_epochs':         [100],
# }

class MyGridSearch(MyBaseSearch):
    def __init__(self, params, num_splits=3):
        super().__init__(params, num_splits)
        self.state_fn = '/data/models/model_selection_grid'

    def fit(self, dataset, verbose=False):
        """ Avalia o desempenho para cada combinação de parâmetros. Usa 'itertools.product' 
        para obter o produto cartesiano entre as listas especificadas em 'self.parameters'.
        Nota: 'itertools.product(A, B)' retorna o mesmo que '((x, y) for x in A for y in B)'.
        """
        self.parameter_sets = []
        self.scores = []
        keys = self.parameters.keys()
        for it, x in enumerate(itertools.product(*params.values())):
            print('Iteraction {:2d}:'.format(it), end=' ')
            t0 = time.time()
            p = dict(zip(keys, x))
            self.parameter_sets.append(p)
            
            split_scores = self.do_cross_validation(p, dataset, verbose)
            self.scores.append(split_scores)
            print('[{:.1f} s]'.format(time.time() - t0))        
        self.show_results()


## Random Searching

In [8]:
params = {
    'model_class':        [Model],
    'loss_class':         ['MSELoss'],
    'optim_class':        ['RMSprop'],
    'optim_lr':           st.uniform(0.001, 0.002),       # range: 0.001-0.003
    'optim_weight_decay': st.uniform(0.0005, 0.0095),     # 0.0005-0.01
    'optim_alpha':        st.uniform(0.7, 0.29),          # 0.7-0.99
    'batch_size':         [30],
    'num_epochs':         [200],
}
rs = MyRandomizedSearch(params, num_splits=5, num_iter=30)
rs.fit(train_ds)

Iteraction  0:  6.24696  18.56860  12.26453  47.52807  14.15301 [87.4 s]
Iteraction  1:  5.31770  13.95588  14.40160  50.43629  13.08523 [83.2 s]
Iteraction  2:  6.26278  20.23700  8.05917  52.62885  16.25098 [83.3 s]
Iteraction  3:  6.33826  20.27767  11.93542  43.58981  13.51593 [83.3 s]
Iteraction  4:  6.92073  19.04859  13.12680  44.50853  10.55239 [83.7 s]
Iteraction  5:  6.03344  17.91357  12.44797  48.98238  13.31631 [83.4 s]
Iteraction  6:  5.99609  13.97222  9.14008  48.35522  14.29169 [83.5 s]
Iteraction  7:  6.06414  17.97815  13.82636  47.17277  11.67926 [83.7 s]
Iteraction  8:  6.28640  19.82875  15.16874  45.19716  11.80703 [83.6 s]
Iteraction  9:  6.15534  18.31290  11.44172  47.54664  10.79677 [83.6 s]
Iteraction 10:  6.08410  12.36792  12.00369  51.14266  15.24391 [83.5 s]
Iteraction 11:  6.68842  21.08453  13.86811  46.90276  12.33577 [83.4 s]
Iteraction 12:  6.00180  12.18465  12.87174  48.67128  13.61852 [83.5 s]
Iteraction 13:  6.82717  22.31982  11.34131  44.86667

In [9]:
batch_size = rs.parameter_sets[rs.best_index]['batch_size']
dloader = DataLoader(train_ds, batch_size=batch_size)
rs.best_trainer.fit_loader(500, dloader)
rs.best_trainer.load_state(rs.state_fn)
rs.best_trainer.evaluate_loader(dloader)

evaluate: 16/16 ok


{'losses': 2.2148042936098906}

## Grid Searching

In [10]:
params = {
    'model_class':        [Model],
    'loss_class':         ['MSELoss'],
    'optim_class':        ['RMSprop'],
    'optim_lr':           [0.001, 0.002, 0.003],
    'optim_weight_decay': [0.0005, 0.005, 0.01],
    'optim_alpha':        [0.7, 0.99],
    'batch_size':         [30],
    'num_epochs':         [200],
}
gs = MyGridSearch(params, num_splits=5)
gs.fit(train_ds)

Iteraction  0:  7.02916  19.48315  11.58768  46.30515  9.29859 [83.1 s]
Iteraction  1:  6.12518  20.44003  16.14215  43.68536  11.29524 [82.7 s]
Iteraction  2:  5.25284  21.52369  13.99203  55.15636  10.41153 [83.2 s]
Iteraction  3:  6.24196  16.30851  13.32567  48.13969  11.65188 [83.0 s]
Iteraction  4:  6.84649  21.08872  13.82133  42.17008  12.57196 [83.3 s]
Iteraction  5:  6.46902  12.75646  12.88601  46.52734  12.02584 [83.2 s]
Iteraction  6:  5.95193  13.46878  8.72200  52.48876  13.62923 [82.9 s]
Iteraction  7:  6.11243  16.46185  6.78928  48.38416  11.99127 [82.6 s]
Iteraction  8:  6.34976  18.85601  13.35772  44.94875  10.21562 [82.9 s]
Iteraction  9:  5.37473  14.62576  7.92592  41.57780  11.76086 [82.9 s]
Iteraction 10:  5.66566  15.62877  12.19935  50.26344  14.14203 [82.9 s]
Iteraction 11:  6.76237  14.19914  16.05852  48.85175  13.95611 [82.8 s]
Iteraction 12:  6.28464  18.23318  12.96982  48.88411  13.96927 [82.8 s]
Iteraction 13:  5.73671  14.33412  10.85710  51.05961  

In [11]:
batch_size = gs.parameter_sets[gs.best_index]['batch_size']
dloader = DataLoader(train_ds, batch_size=batch_size)
gs.best_trainer.fit_loader(500, dloader)
gs.best_trainer.load_state(gs.state_fn)
gs.best_trainer.evaluate_loader(dloader)

evaluate: 16/16 ok


{'losses': 2.6628452054125518}

# Exercícios

1. Adicione um parâmetro adicional na escolha do otimizador: 'SGV' e o 'Adam'.
2. Adicione agora um parâmetro na rede, por exemplo trocar a função de ativação de 'ReLU' para 'Sigmoid'.
3. Adicione um parâmetro a sua escolha.